# TP 3 - Vision por Computadora

Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en
Material_TPs/TP3/images a partir del template Material_TPs/TP3/template
1. (4 puntos) Obtener una detección del logo en cada imagen sin falsos positivos
2. (4 puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen
coca_multi.png con el mismo témplate del ítem 1
3. (2 puntos) Generalizar el algoritmo del item 2 para todas las imágenes.
Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza
de la detección.

## Cargar dependencias
Importamos las bibliotecas requeridas

In [137]:
import cv2
import numpy as np
import glob
import os

TEMPL_PATH = "template/pattern.png"
IMAGES_DIR = "images/*"

def draw_boxes(image, boxes, scores, color=(0, 0, 255)):
    img_draw = image.copy()
    for (x1, y1, x2, y2), s in zip(boxes, scores):
        cv2.rectangle(img_draw, (x1, y1), (x2, y2), color, 2)
        text = f"{s:.3f}"
        cv2.putText(img_draw, text, (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    return img_draw

# -------------------------------------------------------
# 1) Template original en escala de grises
# -------------------------------------------------------
template_gray = cv2.imread(TEMPL_PATH, cv2.IMREAD_GRAYSCALE)
if template_gray is None:
    raise FileNotFoundError(f"No pude leer el template en {TEMPL_PATH}")

th0, tw0 = template_gray.shape[:2]
print("Template original:", th0, tw0)

# Bordes del template
template_edges_orig = cv2.Canny(template_gray, 100, 200)

# -------------------------------------------------------
# 2) Matching sobre bordes (Canny) – para imágenes con color
# -------------------------------------------------------
def match_edges_with_resized_template(gray, template_edges_base):
    img_h, img_w = gray.shape[:2]
    img_edges = cv2.Canny(gray, 100, 200)

    t_h, t_w = template_edges_base.shape[:2]
    templ_edges_used = template_edges_base

    # Si el template es más grande que la imagen → reescalar
    if img_h < t_h or img_w < t_w:
        scale = min(img_w / t_w, img_h / t_h)
        new_w = max(1, int(t_w * scale))
        new_h = max(1, int(t_h * scale))
        templ_edges_used = cv2.resize(template_edges_base, (new_w, new_h),
                                      interpolation=cv2.INTER_AREA)
        t_h, t_w = templ_edges_used.shape[:2]

    res = cv2.matchTemplate(img_edges, templ_edges_used, cv2.TM_CCOEFF_NORMED)
    return res, t_h, t_w

# -------------------------------------------------------
# 3) Matching en gris multiescala – para imágenes retro/B&N
# -------------------------------------------------------
def match_gray_multiscale(gray, template_gray_base, scales):
    img_h, img_w = gray.shape[:2]
    t_h0, t_w0 = template_gray_base.shape[:2]

    max_val_best = -1.0
    max_loc_best = None
    t_h_best, t_w_best = t_h0, t_w0

    for s in scales:
        new_w = int(t_w0 * s)
        new_h = int(t_h0 * s)

        # Descartar escalas ridículas o fuera de límites
        if new_w < 5 or new_h < 5:
            continue
        if new_w > img_w or new_h > img_h:
            continue

        templ_s = cv2.resize(template_gray_base, (new_w, new_h),
                             interpolation=cv2.INTER_AREA)

        res = cv2.matchTemplate(gray, templ_s, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, max_loc = cv2.minMaxLoc(res)

        if max_val > max_val_best:
            max_val_best = max_val
            max_loc_best = max_loc
            t_h_best, t_w_best = new_h, new_w

    return max_val_best, max_loc_best, t_h_best, t_w_best

# -------------------------------------------------------
# 4) Proceso principal – Punto 1
# -------------------------------------------------------
SAT_THRESHOLD =30
GRAY_SCALES = np.linspace(0.7, 1.8, 12)

output_dir = "resultados_tp3_punto1"
os.makedirs(output_dir, exist_ok=True)

image_paths = sorted(glob.glob(IMAGES_DIR))

for img_path in image_paths:
    img = cv2.imread(img_path)
    if img is None:
        print("No pude leer", img_path)
        continue

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Saturación para distinguir color vs B/N
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    sat_mean = hsv[..., 1].mean()

    nombre = os.path.basename(img_path)
    print(f"{nombre}: saturación media = {sat_mean:.1f}")

    if sat_mean < SAT_THRESHOLD:
        # Imagen retro/B&N
        metodo = "GRAY-MULTISCALE"
        max_val, max_loc, th_used, tw_used = match_gray_multiscale(
            gray, template_gray, GRAY_SCALES)
    else:
        # Imagen a color
        metodo = "CANNY"
        res, th_used, tw_used = match_edges_with_resized_template(
            gray, template_edges_orig)
        _, max_val, _, max_loc = cv2.minMaxLoc(res)

    print(f"  -> método = {metodo}, score = {max_val:.3f}")

    top_left = max_loc
    bottom_right = (top_left[0] + tw_used, top_left[1] + th_used)

    boxes = np.array([[top_left[0], top_left[1],
                       bottom_right[0], bottom_right[1]]])
    scores = np.array([max_val])

    img_det = draw_boxes(img, boxes, scores)

    base, ext = os.path.splitext(nombre)
    out_name = os.path.join(output_dir,
                            f"{base}_p1{ext or '.png'}")
    cv2.imwrite(out_name, img_det)
    print("   -> guardado en", out_name)


Template original: 175 400
COCA-COLA-LOGO.jpg: saturación media = 120.3
  -> método = CANNY, score = 0.058
   -> guardado en resultados_tp3_punto1/COCA-COLA-LOGO_p1.jpg
coca_logo_1.png: saturación media = 136.8
  -> método = CANNY, score = 0.110
   -> guardado en resultados_tp3_punto1/coca_logo_1_p1.png
coca_logo_2.png: saturación media = 156.0
  -> método = CANNY, score = 0.077
   -> guardado en resultados_tp3_punto1/coca_logo_2_p1.png
coca_multi.png: saturación media = 72.9
  -> método = CANNY, score = 0.055
   -> guardado en resultados_tp3_punto1/coca_multi_p1.png
coca_retro_1.png: saturación media = 0.0
  -> método = GRAY-MULTISCALE, score = 0.648
   -> guardado en resultados_tp3_punto1/coca_retro_1_p1.png
coca_retro_2.png: saturación media = 147.6
  -> método = CANNY, score = 0.049
   -> guardado en resultados_tp3_punto1/coca_retro_2_p1.png
logo_1.png: saturación media = 67.0
  -> método = CANNY, score = 0.064
   -> guardado en resultados_tp3_punto1/logo_1_p1.png
